In [96]:
"""
Author: Zaw
Findin the three most deviations of Predicting the LogP of molecules by using MLP
"""
import numpy as np
import pandas as pd
import math
import time
import os
import sys
import warnings
warnings.simplefilter(action='ignore')
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from hyperopt import fmin,tpe,hp,STATUS_OK, Trials
import pickle
start = time.time()

In [97]:
# Data path
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/extract_data.csv"
col_list=['LogP','Exp_RT']
lc_df = pd.read_csv(data_path,usecols=col_list)
col_smile_list=['SMILES','LogP','Exp_RT']
lc_smiles_df = pd.read_csv(data_path,usecols=col_smile_list)

# Remove non_retained molecules
# index=lc_df[lc_df['Exp_RT'] < 180].index
# lc_df.drop(lc_df[lc_df['Exp_RT'] < 180].index,inplace=True)

# Import descriptor file
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/descriptors.csv"
des_df = pd.read_csv(data_path,index_col=0)

# Remove non_retained molecules
#des_df  = des_df.drop(des_df.index[[index]])
#lc_smiles_df = lc_smiles_df.drop(lc_smiles_df[[index]])
# combine two dataframe
data_comb = pd.concat([lc_smiles_df,des_df],axis=1)

In [98]:
data_comb

,SMILES,LogP,Exp_RT,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CCSC(=N)N,0.67,93.5,6.635046,0.206019,6.635046,0.206019,0.378489,104.178,96.114,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...,2.64,687.8,12.990628,-0.335181,12.990628,0.035905,0.760602,414.333,389.133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Oc1cccc2c(O)nccc12,1.73,583.6,9.356759,-0.052407,9.356759,0.052407,0.617197,161.160,154.104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NCCc1ccc(S(=O)(=O)F)cc1,1.05,653.1,12.386283,-4.565776,12.386283,0.311786,0.740582,203.238,193.158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CNC(=O)c1cc2c(cn1)[nH]c1ccccc12,1.33,705.6,11.531255,-0.169277,11.531255,0.169277,0.665746,225.251,214.163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,OCCn1cc(-c2ccc3c(c2)CC/C3=N\O)c(-c2ccncc2)n1,2.05,603.4,9.236427,0.039104,9.236427,0.039104,0.567510,334.379,316.235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2067,CC(C)OC(=O)N1CCC(Oc2ncnc3c2cnn3-c2ccc(S(C)(=O)...,1.59,1061.4,14.683496,-3.539697,14.683496,0.054651,0.550748,477.518,453.326,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,CCC(=O)O[C@]1(C(=O)CO)CC[C@H]2[C@@H]3CCC4=CC(=...,3.73,1059.3,12.960935,-1.190444,12.960935,0.073898,0.724147,402.531,368.259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,O=C(c1cc2ccccc2[nH]1)N1CCCC1,1.70,794.3,12.111258,0.133056,12.111258,0.133056,0.777073,214.268,200.156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
# # drop the smile column which can not be used for training
# data_comb = data_comb.drop(['SMILES'],axis=1)

# # Filling the nan with mean values in des_with_lc
# for col in data_comb:
#         data_comb[col].fillna(data_comb[col].mean(),inplace=True)
# # Remove features with low Variance(threshold <= 0.05)

# data_var = data_comb.var()
# del_feat = list(data_var[data_var <= 0.05].index)
# data_comb.drop(columns=del_feat, inplace=True)

# # Remove features with correlation(threshold >0.95)
# corr_matrix = data_comb.corr().abs()
# mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
# tri_df = corr_matrix.mask(mask)
# to_drop =  [ c for c in tri_df.columns if any(tri_df[c] > 0.95)]
# data = data_comb.drop(to_drop,axis=1)

In [100]:
# parameters
hidden_layer_sizes_list = [(150,100,50), (120,80,40) ,(100,50,30)]
max_iter_list =[5,10,50,100,200]
activation_list = ['relu', 'tanh', 'logistic']
solver_list = ['sgd','adam']
alpha_list = [0.0001, 0.05]
learning_rate_list =['constant','adaptive']

In [101]:
# The dataset with lc informations
#load the model with lc
path = os.getcwd()
dirname = os.path.dirname(path)

# load the model
file_path = dirname+"/logP/hyperparameters/mlp_lc.pkl"
best_parameters_with_lc = pickle.load(open(file_path,'rb'))

In [121]:
#Single random run with RT
data_set_1 = des_df
data_set_2 = lc_df
des_with_lc = pd.concat([data_set_1,data_set_2],axis=1)
des_with_lc_feat_corr = des_with_lc.columns[des_with_lc.corrwith(des_with_lc['LogP']) >=0.90][:-1]
des_with_lc = des_with_lc.drop(columns=des_with_lc_feat_corr)

# Rescale the feature
def scale(col):
    return (col - np.min(col))/(np.max(col)-np.min(col))

# Filling the nan with mean values in des_with_lc
for col in des_with_lc:
    des_with_lc[col].fillna(des_with_lc[col].mean(),inplace=True)

# Remove columns with zero vlues
des_with_lc = des_with_lc.loc[:,(des_with_lc**2).sum() != 0]
data = des_with_lc.drop(['LogP'],axis=1)

# Remove features with low Variance(threshold<=0.05)
data_var = data.var()
del_feat = list(data_var[data_var <= 0.05].index)
data.drop(columns=del_feat, inplace=True)

# Remove features with correlation(threshold > 0.95)
corr_matrix = data.corr().abs()
mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
tri_df = corr_matrix.mask(mask)
to_drop =  [ c for c in tri_df.columns if any(tri_df[c] > 0.95)]
data = data.drop(to_drop,axis=1)

# Scale the features
cols = list(data)
data[cols] = data[cols].apply(scale,axis=0)
data = pd.concat([data,des_with_lc['LogP']],axis=1)

# Single random run

# data set preparation
train , test = train_test_split(data,train_size = 0.5,shuffle=True)
#validate , test = train_test_split(rest, train_size = 0.5, shuffle=True)

# training set
data_tra_x = train.drop(['LogP'],axis=1)
data_tra_y = train['LogP']

# # validation set
# data_val_x = validate.drop(['LogP'],axis=1)
# data_val_y = validate['LogP']

# test set
data_tes_x = test.drop(['LogP'],axis=1)
data_tes_y = test['LogP']

best_model = MLPRegressor(hidden_layer_sizes = hidden_layer_sizes_list[best_parameters_with_lc['hidden_layer_sizes']],
                            max_iter = max_iter_list[best_parameters_with_lc['max_iter']],
                            activation = activation_list[best_parameters_with_lc['activation']],
                            solver = solver_list[best_parameters_with_lc['solver']],
                            alpha = alpha_list[best_parameters_with_lc['alpha']],
                            learning_rate = learning_rate_list[best_parameters_with_lc['learning_rate']]
                            )

best_model.fit(data_tra_x,data_tra_y)

# training error
tra_pred = best_model.predict(data_tra_x)
tra_results = ['tra',mean_squared_error(data_tra_y,tra_pred),
            mean_squared_error(data_tra_y,tra_pred,squared=False),
            mean_absolute_error(data_tra_y,tra_pred),
            r2_score(data_tra_y,tra_pred)]

# # validation error
# val_pred = best_model.predict(data_val_x)
# val_results = ['val',mean_squared_error(data_val_y,val_pred),
#             mean_squared_error(data_val_y,val_pred,squared=False),
#             mean_absolute_error(data_val_y,val_pred),
#             r2_score(data_val_y,val_pred)]
# testing error
tes_pred = best_model.predict(data_tes_x)
tes_results = ['tes',mean_squared_error(data_tes_y,tes_pred),
            mean_squared_error(data_tes_y,tes_pred,squared=False),
            mean_absolute_error(data_tes_y,tes_pred),
            r2_score(data_tes_y,tes_pred)]


all_set = tra_results,tes_results


all_set_df = pd.DataFrame(all_set,columns=['set','mse','rmse','mae','r2'])
print(all_set_df)
#all_set_df.to_excel(dirname+"/logP/results/mlp_single_rt.xlsx")
# print('\nSingle random run with LC information is done.')

# run_best_model(des_df,lc_df)

   set       mse      rmse       mae        r2
0  tra  0.201191  0.448543  0.334142  0.929789
1  tes  0.340933  0.583895  0.402618  0.879229


In [122]:
# Predictions
predictions = pd.DataFrame(tes_pred,columns=['Predicted LogP'])

data_test = test
data_test['index'] = data_test.index

data_comb_smiles = data_comb
data_comb_smiles['index'] = data_comb_smiles.index

result = pd.merge(data_test['index'],data_comb_smiles)

test_df = result[['SMILES','LogP']]

In [123]:
test_df

,SMILES,LogP
0,CC(C)c1cc(C(F)(F)F)nc(NNc2nc(C(C)C)cc(C(F)(F)F...,6.55
1,COc1ccccc1N1CCN(C[C@@H](O)COc2cccc3ccccc23)CC1,3.77
2,COc1ccc(/C=C/C(=O)c2c(OC)cc(O)cc2OC)cc1,3.11
3,O=C(O)c1cc2nc(-c3ccc(F)cc3)cc(C(F)(F)F)n2n1,3.67
4,Cc1ccc(-c2nc3ccccc3[nH]2)cc1NC(=O)c1ccco1,3.95
...,...,...
1031,COc1ccc(N(CCC#N)S(=O)(=O)c2cccc3nsnc23)cc1,2.44
1032,O=C1C[C@@H](c2ccc(O)c(O)c2)Oc2c1ccc(O[C@@H]1O[...,-0.39
1033,O=Cc1ccc(O)c2ncccc12,1.54
1034,COc1ccc2c(c1OC)C(=O)O[C@@H]2[C@H]1c2cc3c(cc2CC...,2.74


In [124]:
# create dataframe with predicted LogP and Experimenatal logP
predictions = pd.DataFrame(tes_pred,columns=['Predicted LogP'])

mlp_df = pd.concat([test_df,predictions],axis=1)

mlp_df.rename(columns={'LogP':'Experimental LogP'},inplace=True)

mlp_df['Predicted LogP'] = mlp_df['Predicted LogP'].round(decimals=2)


In [125]:
mlp_df

,SMILES,Experimental LogP,Predicted LogP
0,CC(C)c1cc(C(F)(F)F)nc(NNc2nc(C(C)C)cc(C(F)(F)F...,6.55,4.82
1,COc1ccccc1N1CCN(C[C@@H](O)COc2cccc3ccccc23)CC1,3.77,3.88
2,COc1ccc(/C=C/C(=O)c2c(OC)cc(O)cc2OC)cc1,3.11,3.42
3,O=C(O)c1cc2nc(-c3ccc(F)cc3)cc(C(F)(F)F)n2n1,3.67,2.99
4,Cc1ccc(-c2nc3ccccc3[nH]2)cc1NC(=O)c1ccco1,3.95,3.46
...,...,...,...
1031,COc1ccc(N(CCC#N)S(=O)(=O)c2cccc3nsnc23)cc1,2.44,2.51
1032,O=C1C[C@@H](c2ccc(O)c(O)c2)Oc2c1ccc(O[C@@H]1O[...,-0.39,-0.21
1033,O=Cc1ccc(O)c2ncccc12,1.54,1.32
1034,COc1ccc2c(c1OC)C(=O)O[C@@H]2[C@H]1c2cc3c(cc2CC...,2.74,2.36


In [126]:
# Find deviation
mlp_df['Deviation'] = (mlp_df['Experimental LogP']- mlp_df['Predicted LogP'])**2

In [127]:
# round off decimal value
mlp_df['Deviation'] = mlp_df['Deviation'].round(decimals=2)

In [128]:
mlp_df

,SMILES,Experimental LogP,Predicted LogP,Deviation
0,CC(C)c1cc(C(F)(F)F)nc(NNc2nc(C(C)C)cc(C(F)(F)F...,6.55,4.82,2.99
1,COc1ccccc1N1CCN(C[C@@H](O)COc2cccc3ccccc23)CC1,3.77,3.88,0.01
2,COc1ccc(/C=C/C(=O)c2c(OC)cc(O)cc2OC)cc1,3.11,3.42,0.10
3,O=C(O)c1cc2nc(-c3ccc(F)cc3)cc(C(F)(F)F)n2n1,3.67,2.99,0.46
4,Cc1ccc(-c2nc3ccccc3[nH]2)cc1NC(=O)c1ccco1,3.95,3.46,0.24
...,...,...,...,...
1031,COc1ccc(N(CCC#N)S(=O)(=O)c2cccc3nsnc23)cc1,2.44,2.51,0.00
1032,O=C1C[C@@H](c2ccc(O)c(O)c2)Oc2c1ccc(O[C@@H]1O[...,-0.39,-0.21,0.03
1033,O=Cc1ccc(O)c2ncccc12,1.54,1.32,0.05
1034,COc1ccc2c(c1OC)C(=O)O[C@@H]2[C@H]1c2cc3c(cc2CC...,2.74,2.36,0.14


In [129]:
# The three most deviation values
mlp_df.iloc[list(mlp_df['Deviation'].nlargest(3).index)]

,SMILES,Experimental LogP,Predicted LogP,Deviation
407,CC(=O)OCCn1c([N+](=O)[O-])cnc1/C=[N+](\[O-])C1...,-2.91,1.66,20.88
429,O=C1c2ccccc2C(=O)C1c1ccccn1,-2.59,1.85,19.71
815,C[N+]1(C)CCC(OC(=O)C(c2ccccc2)c2ccccc2)CC1,-0.70,2.73,11.76
